# Measuring hydrostatic masses for SDSSRM-XCS galaxy clusters

In this notebook we perform new measurements of hydrostatic mass for a subset of the SDSSRM-XCS volume-limited, temperature error limited, galaxy cluster sample. We also aim to use the masses to construct mass observable relations between mass and various X-ray quantities, as well as mass and the optical quantity richness. These mass measurements are meant as a companion to the [Giles et al. (2022)](https://arxiv.org/abs/2202.11107) paper, which constructed the sample and reported measurements of temperature and luminosity.

In [1]:
import pandas as pd
import numpy as np
from astropy.units import Quantity, UnitConversionError
from astropy.cosmology import LambdaCDM, WMAP9
import matplotlib.pyplot as plt
from typing import Union, List
from shutil import rmtree
import os
from tqdm import tqdm
from warnings import warn

# This adds the directory above to the path, allowing me to import the common functions that I've written in
#  common.py - this just saves me repeating boring code and makes sure its all consistent
import sys
sys.path.insert(0, '..')
from common import xcs3p_colour, xcs_cosmo, direct_comparison_plot, find_lims, fit_comp

import xga
# This just sets the number of cores this analysis is allowed to use
xga.NUM_CORES = 100
# This is a bodge that will only work because xga_output in notebooks has already been defined, XGA
#  will be made to handle this more gracefully at some point
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/')[0]+'notebooks/xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.samples import ClusterSample
from xga.sourcetools.mass import inv_abel_dens_onion_temp
from xga.relations.fit import scaling_relation_odr
from xga.relations.clusters.MT import arnaud_m500
from xga.models.misc import power_law
from xga.exceptions import NoProductAvailableError, ModelNotAssociatedError
from xga.xspec import single_temp_apec


## Reading in sample file and Declaring XGA ClusterSample

Reading in the sample csv file, containing XAPA positions, redshifts, XCS3P $R_{500}$ estimates, and richnesses. This is used to define the XGA ClusterSample, which in turn is what hydrostatic mass measurements are run on. We make sure to turn peak-finding off, as we base our measurements on XAPA positions.

In [2]:
xcs3p = pd.read_csv("../../sample_files/xcs3p_sdssrm_vol_lim_temperr_25%_clusters.csv")
xcs3p_r2500 = pd.read_csv("../../sample_files/xcs3p_sdssrm_r2500_props_nonvol_lim_temperr_100%_clusters.csv")

In [3]:
# Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
ra = xcs3p['xapa_ra'].values
dec = xcs3p['xapa_dec'].values
z = xcs3p['z'].values
# Not using the IAU names in XCS_NAME column, its easier for me to use the name based on redMaPPer ID
n = xcs3p['name'].values
# In kpc
r500 = Quantity(xcs3p['r500'].values, 'kpc')
# Not likely to use richness in this notebook, but I'm putting it in the sample object anyway
r = xcs3p['richness'].values
r_err = xcs3p['richness_err'].values

# Declaring the actual ClusterSample instance for the XCS sample
xcs_srcs = ClusterSample(ra, dec, z, n, r500=r500, richness=r, richness_err=r_err, cosmology=xcs_cosmo, 
                         load_fits=True, use_peak=False, clean_obs=True, clean_obs_reg='r500', 
                         clean_obs_threshold=0.7)

Declaring BaseSource Sample:  48%|████████████████████▋                      | 72/150 [03:38<02:58,  2.29s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/base.py:870: UserWarning: SDSSXCS-408 spectrum /mnt/lustre/projects/astro/general/dt237/mass_papers/XCS-Mass-Paper-I-Analysis/notebooks/xga_output/0502690101/0502690101_mos2_SDSSXCS-408_ra178.82585_dec23.406762_ri0.049541661400259285_ro0.055583327424681146_grpTrue_mincnt5_ident10604555_8_spec.fits cannot be loaded in due to a mismatch in available ancillary files
  warnings.warn("{src} spectrum {sp} cannot be loaded in due to a mismatch in available"
Declaring BaseSource Sample:  78%|████████████████████████████████▊         | 117/150 [06:52<03:02,  5.52s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/base.py:870: UserWarning: SDSSXCS-55 spectrum /mnt/lustre/projects/astro/general/dt237/mass_papers/XCS-Mass-Paper-I-Analysis/notebooks/xga_output/0149880101/0149880101_pn_SDSSXCS-55_ra227.55039_dec33.516138_ri0.08579161

/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-617 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/base.py:1039: UserWarning: A previous annular spectra profile fit for SDSSXCS-617 was not successful, or no matching spectrum has been loaded, so it cannot be read in
  warnings.warn("A previous annular spectra profile fit for {src} was not successful, or no "
Setting up Galaxy Clusters:  35%|███████████████▌                            | 53/150 [03:38<05:39,  3.50s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/base.py:1039: UserWarning: A previous annular spectra profile fit for SDSSXCS-6245 was not successful, or no matching spectrum has been loaded, so it cannot be read in
  warnings.warn("A previ

Setting up Galaxy Clusters:  53%|███████████████████████▍                    | 80/150 [07:57<21:47, 18.67s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/base.py:1039: UserWarning: A previous annular spectra profile fit for SDSSXCS-608 was not successful, or no matching spectrum has been loaded, so it cannot be read in
  warnings.warn("A previous annular spectra profile fit for {src} was not successful, or no "
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-608 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/base.py:1039: UserWarning: A previous annular spectra profile fit for SDSSXCS-608 was not successful, or no matching spectrum has been loaded, so it cannot be read in
  warnings.warn("A previo

Setting up Galaxy Clusters:  80%|██████████████████████████████████▍        | 120/150 [11:25<02:58,  5.96s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/general.py:124: UserWarning: SDSSXCS-382 has not been detected in all region files, so generating and fitting products with the 'region' reg_type will not use all available data
  warnings.warn("{n} has not been detected in all region files, so generating and fitting products"
Setting up Galaxy Clusters:  81%|██████████████████████████████████▉        | 122/150 [11:32<02:07,  4.56s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sources/base.py:1039: UserWarning: A previous annular spectra profile fit for SDSSXCS-5624 was not successful, or no matching spectrum has been loaded, so it cannot be read in
  warnings.warn("A previous annular spectra profile fit for {src} was not successful, or no "
Setting up Galaxy Clusters:  83%|███████████████████████████████████▊       | 125/150 [11:38<01:13,  2.95s/it]/mnt/pact/dt237/code/Pyc

In [95]:
nums = {}
for src in xcs_srcs:
    try:
        annspec = src.get_annular_spectra()
        nums[src.name] = annspec.num_annuli
    except NoProductAvailableError:
        pass
nums

{'SDSSXCS-124': 12,
 'SDSSXCS-2789': 17,
 'SDSSXCS-290': 4,
 'SDSSXCS-1018': 6,
 'SDSSXCS-134': 10,
 'SDSSXCS-119': 4,
 'SDSSXCS-209': 9,
 'SDSSXCS-15': 7,
 'SDSSXCS-71': 14,
 'SDSSXCS-5020': 4,
 'SDSSXCS-2346': 2,
 'SDSSXCS-586': 4,
 'SDSSXCS-11331': 6,
 'SDSSXCS-28849': 4,
 'SDSSXCS-31144': 4,
 'SDSSXCS-239': 8,
 'SDSSXCS-395': 9,
 'SDSSXCS-944': 8,
 'SDSSXCS-593': 8,
 'SDSSXCS-35404': 4,
 'SDSSXCS-7190': 4,
 'SDSSXCS-7783': 8,
 'SDSSXCS-5743': 4,
 'SDSSXCS-1131': 4,
 'SDSSXCS-29799': 4,
 'SDSSXCS-852': 12,
 'SDSSXCS-5830': 4,
 'SDSSXCS-5977': 4,
 'SDSSXCS-16227': 4,
 'SDSSXCS-2984': 4,
 'SDSSXCS-6955': 4,
 'SDSSXCS-64824': 4,
 'SDSSXCS-8366': 4,
 'SDSSXCS-120': 12,
 'SDSSXCS-572': 9,
 'SDSSXCS-226': 9,
 'SDSSXCS-22': 15,
 'SDSSXCS-103': 12,
 'SDSSXCS-212': 4,
 'SDSSXCS-19922': 4,
 'SDSSXCS-890': 4,
 'SDSSXCS-31': 13,
 'SDSSXCS-325': 4,
 'SDSSXCS-26424': 4,
 'SDSSXCS-5159': 12,
 'SDSSXCS-9803': 5,
 'SDSSXCS-1884': 4,
 'SDSSXCS-5': 14,
 'SDSSXCS-13': 10,
 'SDSSXCS-5225': 5,
 'SDSSXCS-

In [4]:
# This cluster is causing some problems for an unknown reason, but will be restored later when I've figured out why
# del xcs_srcs['SDSSXCS-408']

## Hydrostatic Mass Measurement

Here we measure hydrostatic mass profiles for clusters in the SDSSRM-XCS sample, using $S_{B}$ profiles to calculate density profiles and combining them with temperature profiles to measure hydrostatic mass. 

We fit double beta models to the PSF-corrected $S_B$ profiles, from which we infer density profiles and fit simplified Vikhlinin density models to them. Temperature profiles are measured from annular spectra, deprojected, and then fitted with simplified Vikhlinin temperature models.

In this instance we choose to generate profiles out to the $R_{500}$ value measured by Giles et al. 2022. Each annulus is required to have a total of 1500 counts in each annulus when the annulus is applied to the combined observations.

In [ ]:
sb_dblbeta_simpvikh_1_totmc1500_mp = inv_abel_dens_onion_temp(xcs_srcs, xcs_srcs.r500, 'double_beta', 
                                                                'simple_vikhlinin_dens', 'simple_vikhlinin_temp', 
                                                                'r500', temp_annulus_method='min_cnt', 
                                                                temp_min_cnt=Quantity(1500, 'ct'), 
                                                                temp_use_combined=True)

Running XSPEC Fits: 100%|███████████████████████████████████████████████████████| 1/1 [00:19<00:00, 19.95s/it]
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/mass.py:51: UserWarning: The global temperature fit for SDSSXCS-6307 has failed, and as such we're very unlikely to be able to measure a mass and we're not even going to try.
  warn("The global temperature fit for {} has failed, and as such we're very unlikely to be able to measure "
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/temperature.py:291: UserWarning: The requested annuli for SDSSXCS-290 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  warn("The requested annuli for {s} cannot be created, the data quality is too low. As such a set "
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/temperature.py:291: UserWarning: The requested annuli for SDSSXCS-119 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  wa

/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/temperature.py:291: UserWarning: The requested annuli for SDSSXCS-1884 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  warn("The requested annuli for {s} cannot be created, the data quality is too low. As such a set "
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/temperature.py:291: UserWarning: The requested annuli for SDSSXCS-23 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  warn("The requested annuli for {s} cannot be created, the data quality is too low. As such a set "
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/temperature.py:291: UserWarning: The requested annuli for SDSSXCS-617 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  warn("The requested annuli for {s} cannot be created, the data quality is too low. As such a set "
/mnt/pact/dt237/code/PycharmProjec

/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/temperature.py:291: UserWarning: The requested annuli for SDSSXCS-7432 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  warn("The requested annuli for {s} cannot be created, the data quality is too low. As such a set "
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/temperature.py:291: UserWarning: The requested annuli for SDSSXCS-11613 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  warn("The requested annuli for {s} cannot be created, the data quality is too low. As such a set "
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/temperature.py:291: UserWarning: The requested annuli for SDSSXCS-5939 cannot be created, the data quality is too low. As such a set of four annuli will be returned
  warn("The requested annuli for {s} cannot be created, the data quality is too low. As such a set "
/mnt/pact/dt237/code/PycharmPr

Running XSPEC Fits: 100%|███████████████████████████████████████████████▊| 1106/1109 [34:56<10:36, 212.07s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/xspec/run.py:59: UserWarning: An XSPEC fit for SDSSXCS-21847 has timed out
  warnings.warn("An XSPEC fit for {} has timed out".format(source_name))
Running XSPEC Fits: 100%|█████████████████████████████████████████████▉| 1107/1109 [1:02:07<21:16, 638.02s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/xspec/run.py:59: UserWarning: An XSPEC fit for SDSSXCS-21847 has timed out
  warnings.warn("An XSPEC fit for {} has timed out".format(source_name))
Running XSPEC Fits: 100%|█████████████████████████████████████████████▉| 1108/1109 [1:02:08<07:26, 446.85s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/xspec/run.py:59: UserWarning: An XSPEC fit for SDSSXCS-21847 has timed out
  warnings.warn("An XSPEC fit for {} has timed out".format(source_name))
Running XSPEC Fits: 100%|███████████████████████████████████████████████| 1109/1109 [1:02:0

The chain is shorter than 50 times the integrated autocorrelation time for 6 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 400;
tau: [ 746.76270404  901.02431188 1664.97523285 1486.3947845  1551.48875216
 1635.78633931]
The chain is shorter than 50 times the integrated autocorrelation time for 5 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 400;
tau: [785.15830836 305.30235483 655.47334201 752.39408518 784.42562666
 604.7694877 ]


...

The chain is shorter than 50 times the integrated autocorrelation time for 1 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 400;
tau: [239.75625622 244.79400541 826.03773634 254.260189   151.51616397
 252.14558986]
The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 400;
tau: [454.34351669 609.14150335 422.14559474 505.57645702 247.91246098
 265.56419747]
The chain is shorter than 50 times the integrated autocorrelation time for 1 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 400;
tau: [  84.30768494   86.39840641 1063.10177526   92.46393395   93.94192334
   90.17067348]


Generating SDSSXCS-2092 hydrostatic mass profile:  97%|██████████████████▎| 145/150 [1:21:37<03:31, 42.26s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/mass.py:240: UserWarning: The density profile for SDSSXCS-2092 could not be generated
  warn("The density profile for {} could not be generated".format(src.name))
Generating SDSSXCS-17923 hydrostatic mass profile:  97%|█████████████████▌| 146/150 [1:21:37<02:49, 42.26s/it]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/sourcetools/mass.py:240: UserWarning: The density profile for SDSSXCS-17923 could not be generated
  warn("The density profile for {} could not be generated".format(src.name))
Generating SDSSXCS-68 hydrostatic mass profile:  98%|████████████████████▌| 147/150 [1:21:37<02:06, 42.26s/it]

The chain is shorter than 50 times the integrated autocorrelation time for 3 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 400;
tau: [478.57120968 471.29247017 324.09308278 552.39224513 283.56223698
 249.85481956]
The chain is shorter than 50 times the integrated autocorrelation time for 4 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 400;
tau: [669.41967817 651.36869907 233.35471736 594.29698452 230.90578636
 430.97462831]


Generating SDSSXCS-147 hydrostatic mass profile:  99%|███████████████████▋| 148/150 [1:22:26<00:57, 28.74s/it]

The chain is shorter than 50 times the integrated autocorrelation time for 3 parameter(s). Use this estimate with caution and run a longer chain!
N/50 = 400;
tau: [293.36414555 877.58024798 749.54970173 569.86668615 329.25485639
 275.07516415]


In [ ]:
# print("There are", len([m for m in sb_dblbeta_simpvikh_11_m500[:, 0] if np.isfinite(m)]), 
#       "clusters with successful mass measurements")

In [10]:
sb_dblbeta_simpvikh_1_mc1500_m500 = []
# Tidy this up
gmasses = []

with tqdm(desc='Measuring Masses for SDSSRM-XCS Clusters', total=len(sb_dblbeta_simpvikh_1_totmc1500_mp)) as onwards:
    for mp in sb_dblbeta_simpvikh_1_totmc1500_mp:
        if mp is not None:
            rel_src = xcs_srcs[mp.src_name]
            m = mp.mass(rel_src.r500)[0]
            gm = mp.density_profile.gas_mass('simple_vikhlinin_dens', rel_src.r500)[0]
        else:
            m = Quantity([np.NaN, np.NaN, np.NaN], 'Msun')
            gm = Quantity([np.NaN, np.NaN, np.NaN], 'Msun')
            
        sb_dblbeta_simpvikh_1_mc1500_m500.append(m)
        gmasses.append(gm)
        
        onwards.update(1)

sb_dblbeta_simpvikh_1_mc1500_m500 = Quantity(sb_dblbeta_simpvikh_1_mc1500_m500)
gmasses = Quantity(gmasses)

Measuring Masses for SDSSRM-XCS Clusters:   0%|                                       | 0/150 [00:00<?, ?it/s]/mnt/pact/dt237/code/PycharmProjects/XGA/xga/products/profile.py:475: UserWarning: The outer radius you supplied is greater than or equal to the outer radius covered by the data, so you are effectively extrapolating using the model.
  warn("The outer radius you supplied is greater than or equal to the outer radius covered by the data, so"
Measuring Masses for SDSSRM-XCS Clusters: 100%|█████████████████████████████| 150/150 [10:55<00:00,  4.37s/it]


In [ ]:
# all_info = []
# for src_ind, src in enumerate(xcs_srcs):
#     mass = sb_dblbeta_simpvikh_1_mc1500_m500[src_ind, :].value
    
#     if sb_dblbeta_simpvikh_1_totmc1500_mp[src_ind] is not None:
#         try:
#             alt_r500 = sb_dblbeta_simpvikh_1_totmc1500_mp[src_ind].overdensity_radius(500, src.redshift, src.cosmo)
#             alt_m500 = sb_dblbeta_simpvikh_1_totmc1500_mp[src_ind].mass(alt_r500)[0]
            
#             alt_r2500 = sb_dblbeta_simpvikh_1_totmc1500_mp[src_ind].overdensity_radius(2500, src.redshift, src.cosmo)
#             alt_m2500 = sb_dblbeta_simpvikh_1_totmc1500_mp[src_ind].mass(alt_r2500)[0]
#         except ValueError:
#             alt_m500 = Quantity([np.NaN, np.NaN, np.NaN], 'Msun')
#             alt_m2500 = Quantity([np.NaN, np.NaN, np.NaN], 'Msun')
#     else:
#         alt_m500 = Quantity([np.NaN, np.NaN, np.NaN], 'Msun')
#         alt_m2500 = Quantity([np.NaN, np.NaN, np.NaN], 'Msun')
                
#     rich = src.richness.value
#     z = src.redshift
#     try:
#         tx = src.get_temperature(src.r500).value
#         lx = src.get_luminosities(src.r500, lo_en=Quantity(0.5, 'keV'), hi_en=Quantity(2.0, 'keV')).value
#         lxce = src.get_luminosities(src.r500, inner_radius=src.r500*0.15, lo_en=Quantity(0.5, 'keV'), 
#                                     hi_en=Quantity(2.0, 'keV')).value
#     except ModelNotAssociatedError:
#         tx = np.array([np.NaN, np.NaN, np.NaN])
#         lx = np.array([np.NaN, np.NaN, np.NaN])
#         lxce = np.array([np.NaN, np.NaN, np.NaN])
        
#     all_info.append([z, *mass, *rich, *tx, *lx, *lxce, *alt_m500.value, *alt_m2500.value])

# all_info = np.array(all_info)
# all_info_df = pd.DataFrame(data=all_info, columns=['z', 'm500', 'm500-', 'm500+', 'richness', 'richness_err', 
#                                                    'Tx500', 'Tx500-', 'Tx500+', 'Lx500', 'Lx500-', 'Lx500+', 
#                                                    'Lxce500', 'Lxce500-', 'Lxce500+', 'alt_m500', 'alt_m500-', 
#                                                    'alt_m500+', 'alt_m2500', 'alt_m2500-', 'alt_m2500+'])
# all_info_df.to_csv('prelim_masses_xapa_centres.csv', index=False)

In [12]:
sdss_cols = ['ra', 'dec', 'z', 'name', 'Tx', 'Tx-', 'Tx+', 'Lx', 'Lx-', 'Lx+', 'Lxb', 'Lxb-', 'Lxb+', 
            'Mhy', 'Mhy-', 'Mhy+', 'Mg', 'Mg-', 'Mg+', 'rich', 'rich_err']
sdss_data = np.hstack([xcs_srcs.ra_decs.value, xcs_srcs.redshifts.T[..., None], xcs_srcs.names.T[..., None], 
                           xcs_srcs.Tx(quality_checks=False).value, 
                           xcs_srcs.Lx(xcs_srcs.r500, quality_checks=False).value, 
                          xcs_srcs.Lx(xcs_srcs.r500, quality_checks=False, lo_en=Quantity(0.01, 'keV'), 
                                        hi_en=Quantity(100, 'keV')).value, 
                       sb_dblbeta_simpvikh_1_mc1500_m500.to('1e+14 Msun').value, 
                       gmasses.to('1e+14 Msun').value, xcs_srcs.richness.value])
sdss_df = pd.DataFrame(sdss_data, columns=sdss_cols)
sdss_df.to_csv('thesis_sdss_meas.csv', index=False)
sdss_df.head(10)

/mnt/pact/dt237/code/PycharmProjects/XGA/xga/samples/extended.py:479: UserWarning: Those spectra have no associated XSPEC fit to SDSSXCS-6307
  warn(str(err))
/mnt/pact/dt237/code/PycharmProjects/XGA/xga/samples/base.py:280: UserWarning: These spectra have no associated XSPEC fit to SDSSXCS-6307.
  warn(str(err))


,ra,dec,z,name,Tx,Tx-,Tx+,Lx,Lx-,Lx+,...,Lxb-,Lxb+,Mhy,Mhy-,Mhy+,Mg,Mg-,Mg+,rich,rich_err
0,0.80057775,-6.0918182,0.2474828,SDSSXCS-124,6.92015,0.1163663749999992,0.1116456080000007,5.31039497e+44,2.66229592574526e+42,2.4917862502903803e+42,...,1.9472816076054843e+43,1.9727787831181053e+43,7.871461560565212,0.8315951962102863,0.8170935136556389,1.0497754408987432,0.002671203662080157,0.002677324929883438,109.550186,4.4896803
1,0.95553986,2.068019,0.105285384,SDSSXCS-2789,4.53982,0.0730187390000001,0.074565926,1.029585351e+44,5.9013117083831006e+41,6.4802489310322005e+41,...,3.6500470323010505e+42,3.5337758321586305e+42,5.305239234623756,0.769379385611082,0.8162252047987939,0.2776333161704526,0.0011944057560014847,0.0012060626664146487,38.904396000000006,2.8302057
2,2.7226392,29.161021,0.34849474,SDSSXCS-290,5.31695,0.2978612860000007,0.3363328149999995,2.856684648e+44,6.287688268367601e+42,5.761349886177681e+42,...,3.8814952482348093e+43,3.677474692465036e+43,3.0975910251578815,0.3826599999065154,0.3908021253239738,0.4870513623059551,0.0027624991008549224,0.0026620737390731252,105.09573,5.9943213
3,4.4063253,-0.87619162,0.21440332,SDSSXCS-1018,3.90154,0.1435351899999997,0.1515063040000006,8.044819691000001e+43,1.3301261795678301e+42,1.4429986532833002e+42,...,5.91709700237795e+42,5.5513793421503903e+42,3.5445171559147917,0.5806382140918169,0.5355195670608626,0.2619308180201717,0.002086735159660391,0.002161842676800977,56.996796,3.2192018
4,4.9083898,3.6098177,0.27730444,SDSSXCS-134,6.72636,0.1165286779999998,0.1165610109999999,4.8118889700000005e+44,2.5979896055238903e+42,2.5333202927755703e+42,...,1.941465344835279e+43,1.9757316883910582e+43,5.270972186945933,0.5204600816598332,0.5578078943261007,0.9027082384124043,0.002702996171646719,0.0028627354714790628,108.60438,4.7924843
5,8.464699000000001,-7.8629006,0.3041736,SDSSXCS-119,4.93463,0.400290461,0.4561698349999998,2.049786894e+44,5.98675870665629e+42,6.07824336733596e+42,...,4.1504687959522453e+43,4.024668259058132e+43,nan,nan,nan,nan,nan,nan,128.19176000000002,5.6970525
6,9.2768176,9.1571879,0.26882175,SDSSXCS-209,8.02713,0.1918950019999998,0.2117091880000003,4.4028654710000005e+44,3.9312175025851006e+42,2.2085091380510804e+42,...,2.7819152619000073e+43,2.1261039880213703e+43,9.312829063587843,0.9805979264132864,0.9469651774330526,0.993735860252137,0.001907227722551719,0.0019023319630782816,101.70913,4.746494
7,11.62825,20.467691,0.103966124,SDSSXCS-15,2.5989,0.1290666759999998,0.139564472,3.346000822e+43,8.221842210207401e+41,6.029793305055101e+41,...,2.80764836197768e+42,3.2759733752839103e+42,4.968998190985785,1.1576754211846683,1.187072948872649,0.159568496869965,0.0008463542119589259,0.0008320504403585158,123.355934,3.702013
8,13.996563,26.330304,0.1961017,SDSSXCS-71,5.79498,0.1212898949999994,0.1164601819999999,2.7011214620000004e+44,1.68283744445589e+42,1.58053911640011e+42,...,1.1123645993216431e+43,1.124697395489784e+43,5.484082314335319,0.4491889974406019,0.4375744513915813,1.045923426013181,0.0019086709274671878,0.001865632603873438,112.92153,4.16637
9,16.061785,25.086602,0.16580287,SDSSXCS-5020,3.97149,0.337614571,0.3905836569999996,5.026335576e+43,1.95047342614883e+42,1.66522980016521e+42,...,1.0519458398044761e+43,9.83199704102187e+42,3.994999508747787,1.8944469510403497,2.2483869713864615,0.0827848996292922,0.001828184417388184,0.0017045328285384671,30.468397,3.0966108
